In [ ]:
import sys
sys.path.append("/home/jovyan/.local/lib/python3.8/site-packages")

In [1]:
#Prep pandas df for timeseriescv
import pandas as pd
pdDf=pd.read_csv("CleanedSpy.csv",index_col=0)
pdDf=pdDf.rename(columns={"Time": "pred_times"})
pdDf['eval_times']=pdDf['pred_times']+60;#Change to 300 for 5 min for example

display(pdDf)

,Ticker,pred_times,Volume,logChange,label,prev1,prev2,prev3,prev4,prevVol1,prevVol2,prevVol3,prevVol4,eval_times
0,1,1562170800,117422.0,-0.000101,-0.000168,-0.000134,0.000202,-0.000403,0.000000,50947.0,9238.0,12661.0,100.0,1562170860
1,1,1562171520,1000.0,-0.000168,0.000269,-0.000101,-0.000134,0.000202,-0.000403,117422.0,50947.0,9238.0,12661.0,1562171580
2,1,1562172240,3308.0,0.000269,0.000067,-0.000168,-0.000101,-0.000134,0.000202,1000.0,117422.0,50947.0,9238.0,1562172300
3,1,1562172960,690.0,0.000067,-0.000168,0.000269,-0.000168,-0.000101,-0.000134,3308.0,1000.0,117422.0,50947.0,1562173020
4,1,1562173620,8819.0,-0.000168,-0.000504,0.000067,0.000269,-0.000168,-0.000101,690.0,3308.0,1000.0,117422.0,1562173680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118173,1,1592870040,284753.0,-0.000451,0.000773,0.000515,0.000661,-0.000193,0.000403,525818.0,283778.0,209723.0,657198.0,1592870100
118174,1,1592870100,399185.0,0.000773,0.000169,-0.000451,0.000515,0.000661,-0.000193,284753.0,525818.0,283778.0,209723.0,1592870160
118175,1,1592870160,466966.0,0.000169,-0.000072,0.000773,-0.000451,0.000515,0.000661,399185.0,284753.0,525818.0,283778.0,1592870220
118176,1,1592870220,471539.0,-0.000072,-0.000064,0.000169,0.000773,-0.000451,0.000515,466966.0,399185.0,284753.0,525818.0,1592870280


In [2]:
#USING MLFLOW library instead
from timeseriescv import cross_validation as cv
from sklearn.model_selection import train_test_split
from itertools import tee

train, test=train_test_split(pdDf, test_size=0.2, shuffle=False)
splits=8
crossval=cv.PurgedWalkForwardCV(n_splits=splits+2) #Default settings: n_splits=10, n_test_splits=1, min_train_splits=2, max_train_splits=None
crossval=crossval.split(train, train['label'],train['pred_times'],train['eval_times'],False)
#True above means identical time intervals as opposed to identical sample splits
crossval,crossval_backup=tee(crossval)

In [3]:
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True, False],
    'max_depth': [3,4,5,6],
    'max_features': [8,10,11],
    'min_samples_leaf': [3,4,5],#3,4,5
    'min_samples_split': [8,10,12],#8,10,12
    'n_estimators': [100,200,300,1000]#100,200,300,1000
}


In [4]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

# Create a based model
rf = RandomForestRegressor(random_state=1)
# Instantiate the grid search model
rndm_search = RandomizedSearchCV(
    estimator = rf, param_distributions = param_grid, n_iter=20,cv = crossval, scoring='r2', n_jobs = -1)

In [5]:
import numpy as np
from sklearn.metrics import r2_score
def evaluate(model, test_features, test_labels):
        predictions = model.predict(test_features)
        r2=r2_score(test_labels, predictions)
        print('Model Performance')
        print('r2: {:0.4f}.'.format(r2))
        return r2

In [6]:
import joblib 
from dask.distributed import Client, progress
client = Client()

with joblib.parallel_backend('dask'):
    rndm_search.fit(train[['Ticker','Volume','prevVol1','prevVol2','prevVol3','prevVol4',
                          'logChange','prev1','prev2','prev3','prev4']],train['label'])

In [11]:
import pandas as pd
pd.DataFrame(rndm_search.cv_results_).head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_estimators,param_min_samples_split,param_min_samples_leaf,param_max_features,param_max_depth,param_bootstrap,...,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,mean_test_score,std_test_score,rank_test_score
0,36.595618,7.101749,0.055678,0.023916,100,8,4,8,3,True,...,-0.024310,-0.006681,0.001223,-0.069175,-0.002328,0.010490,-0.007778,-0.015952,0.023470,1
1,98.891027,17.837601,0.074288,0.004168,200,10,3,10,4,True,...,-0.025788,-0.017967,-0.002048,-0.076205,-0.006677,0.011147,-0.009834,-0.022355,0.026729,6
2,227.749120,68.445447,0.112872,0.027493,300,12,5,10,5,False,...,-0.089230,-0.026522,-0.008509,-0.085486,-0.055940,-0.003827,-0.016938,-0.045301,0.033220,18
3,139.209249,41.734276,0.074809,0.008495,200,10,3,10,5,True,...,-0.027537,-0.020815,-0.003922,-0.075379,-0.009255,0.010107,-0.009996,-0.023958,0.026446,7
4,156.457362,47.074498,0.115759,0.033788,300,10,3,10,3,False,...,-0.009995,-0.016887,-0.002610,-0.119150,-0.017675,0.012874,-0.054265,-0.039451,0.046372,13


In [ ]:
print("Best parameters: ")
print(rndm_search.best_params_)
best_grid = rndm_search.best_estimator_
print("\nTest")
grid_accuracy = evaluate(best_grid, test[['Ticker','Volume','prevVol1','prevVol2','prevVol3','prevVol4',
                          'logChange','prev1','prev2','prev3','prev4']], test['label'])
print("\nTrain")
grid_accuracy = evaluate(best_grid, train[['Ticker','Volume','prevVol1','prevVol2','prevVol3','prevVol4',
                          'logChange','prev1','prev2','prev3','prev4']], train['label'])

In [ ]:
import matplotlib.pyplot as plt

#Obtain matrix with rows = fold #, cols=train/test, 
data=[]
for i in range(splits):
    train_temp,test_temp=next(crossval_backup)
    data.append([train_temp[-1]/len(pdDf),(test_temp[1]-train_temp[-1])/len(pdDf),test_temp[-1]/len(pdDf)])
#Need to take transpose of data
data=list(map(list,zip(*data)))
#Need to subtract training from test
import operator
data[2]=list(map(operator.sub, data[2], data[0]))
data[2]=list(map(operator.add, data[2], data[1]))

rows=('Train','Purge','Test')
columns=['Fold %d'%x for x in range(1,splits+1)]
#columns.append("Holdout")
values=np.arange(0,1,0.1)

colors = plt.cm.BuPu([0.25,0,0.5])
n_rows = len(data)

index = np.arange(len(columns)) + 0.3
bar_width = 0.4

# Initialize the vertical-offset for the stacked bar chart.
y_offset = np.zeros(len(columns))

# Plot bars and create text labels for the table
cell_text = []
for row in range(n_rows):
    plt.bar(index, data[row], bar_width, bottom=y_offset, color=colors[row])
    y_offset = y_offset + data[row]
    cell_text.append(['%1f' % x for x in y_offset])
# Reverse colors and text labels to display the last value at the top.
colors = colors[::-1]
cell_text.reverse()

# Add a table at the bottom of the axes
the_table = plt.table(cellText=cell_text,
                      rowLabels=rows,
                      rowColours=colors,
                      colLabels=columns,
                      loc='bottom')

# Adjust layout to make room for the table:
plt.subplots_adjust(left=0.2, bottom=0.2)

plt.ylabel("Cross Validation %")
plt.yticks(values, ['%1.1f' % val for val in values])
plt.xticks([])
plt.title('Cross Validation Train/Test split')

plt.show()

In [ ]:
#TODO:
#Add more stocks
#Compare via non k-fold? (or other k-fold types?)

In [ ]:
test[['Ticker','Volume','prevVol1','prevVol2','prevVol3','prevVol4',
                          'logChange','prev1','prev2','prev3','prev4']]



In [ ]:
import cloudpickle as cp
from urllib.request import urlopen
model = cp.load(urlopen("https://raw.githubusercontent.com/grantjensen/Backtesting2/master/myModel.cpickle"))

In [ ]:
model.predict([[1, 0,0,0,0,0,1,1,1,1,1]])

In [14]:
pkl_filename = "myModel2.cpickle"
with open(pkl_filename, 'wb') as file:
    pickle.dump(rndm_search.best_estimator_, file)

NameError: name 'rnmd_search' is not defined

In [13]:
pickle.dump( rndm_search, open( "myModel2.pickle", "wb" ) )

TypeError: cannot pickle 'generator' object